In [ ]:
from pathlib import Path
import sys  

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

# VMFI Data processing pipeline

This workbook aims to emulate the current data processing pipeline that occurs in VMFI pipeline. The logic and processing is largely based on the following document [Insights data portal - Data sources and sql analysis](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) and will stay true to this document even if the existing stored procedures are doing something different. This will form the basis of a gap analysis going forward. 

All data loaded in the following workbook comes from the set of CSV files in the `data` folder alongside this workbook. These datasets are for the most part from the list at the start of the linked document. However, because there is additional standing data required to fully implement the pipeline then this data has been exported from the development VMFI pipeline database. These files are currently: 

| File name | DB Table |
|:----------|----------|
|standing_data_cdc.csv | standing_data.cdc |

In [ ]:
import src.pipeline.pre_processing as pre_processing
import pandas as pd
import numpy as np
import time
import glob
import os

In [ ]:
# Create and clean directory
from pathlib import Path
Path("output/pre-processing").mkdir(parents=True, exist_ok=True)

# files = glob.glob("output/pre-processing/*")
# for f in files:
#     os.remove(f)

In [ ]:
start_time = time.time()
current_year = 2022

## CDC data load and preparation

School buildings condition dataset. Based on the surveys performed throughout 2018-2019.

The data in the file `data/standing_data_cdc.csv` is just an export of the data in `standing_data.cdc` table. Without the Year and Import ID fields. In future this will likely have to be read directly from the source database as per [this document.](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) 

In [ ]:
cdc = pre_processing.prepare_cdc_data('data/cdc.csv', current_year)

In [ ]:
#cdc.to_csv('output/pre-processing/cdc.csv')
cdc

## School Census data load

*Pupil Census* - DfE data collection providing information about school and pupil characteristics, for example percentage of pupils claiming free school`z meals, or having English as their second language. 

*Workforce census* - Single reference for all school workforce statistics based on staff working in publicly funded schools in England.

The following code loads both the workforce and pupil census data and preforms an `inner` join by URN on the data sets.

In [ ]:
census = pre_processing.prepare_census_data('data/census_workforce.xlsx', 'data/census_pupils.csv')


In [ ]:
#census.to_csv('output/pre-processing/census.csv')
census

## Special Education Needs (SEN) data load and preparation

Special educational needs dataset. Contains information about the number of pupils, who require various SEN provisions. This loads the `SEN` data, which originates from [here](https://explore-education-statistics.service.gov.uk/find-statistics/special-educational-needs-in-england#dataDownloads-1)

In [ ]:
sen = pre_processing.prepare_sen_data('data/sen.csv')

In [ ]:
#sen.to_csv("output/pre-processing/sen.csv")
sen

## KS2 and KS4 processing

In [ ]:
ks2 = pre_processing.prepare_ks2_data('data/ks2.xlsx')

In [ ]:
#ks2.to_csv('output/pre-processing/ks2.csv')
ks2

In [ ]:
ks4 = pre_processing.prepare_ks4_data('data/ks4.xlsx')

In [ ]:
#ks4.to_csv('output/pre-processing/ks4.csv')
ks4

## AR Data load and preparation

This loads the Annual accounts return dataset and the corresponding mapping file. This extract only contains benchmarking section, which consists of submissions of costs, income, and balances of individual academies.

The mapping file, contains the mapping from AR4 cell references to cost categories and descriptions.

In [ ]:
academy_ar = pre_processing.prepare_aar_data('data/academy_ar.xlsx')
central_services = pre_processing.prepare_central_services_data('data/academy_ar.xlsx')

In [ ]:
#academy_ar.to_csv('output/pre-processing/academy_ar.csv')
academy_ar

In [ ]:
central_services

Create a summary table for the AR stance of each distinct academy in the table.

Now compute the trust financial position in the same manor as the individual academy position

## Academy and maintained schools data load and preparation

This reads the main GIAS data (edubasealldataYYYYMMDD file) and the associated links file (links_edubasealldataYYYYMMDD file). This is taken from the [GIAS Service](https://get-information-schools.service.gov.uk/help)

Other columns are tidied up by asserting the correct type for that column. This is tidying phase is largly because on load integer columns will be inferred to be a float as opposed to an integer.

In [ ]:
schools = pre_processing.prepare_schools_data('data/gias.csv','data/gias_links.csv')


In [ ]:
#schools.to_csv('output/pre-processing/schools.csv')
schools.sort_index()

In [ ]:
cfo = pre_processing.build_cfo_data('data/cfo.xlsx')

Merge required GIAS, census, sen, cdc, PFI, and arr data with the base academy data

In [ ]:
import src.pipeline.input_schemas as input_schemas
import src.pipeline.mappings as mappings
import src.pipeline.config as config
import datetime


def build_academy_data(
    academy_data_path,
    links_data_path,
    year,
    schools,
    census,
    sen,
    cdc,
    aar,
    ks2,
    ks4,
    cfo,
    central_services,
):
    accounts_return_period_start_date = datetime.date(year - 1, 9, 10)
    academy_year_start_date = datetime.date(year - 1, 9, 1)
    academy_year_end_date = datetime.date(year, 8, 30)
    print('here')
    academies_list = pd.read_csv(
        academy_data_path,
        encoding="utf8",
        usecols=input_schemas.academy_master_list.keys(),
    )
    print('here')
    academies_list.replace(to_replace={'DNS':np.nan,'n/a':np.nan}, inplace=True)
    academies_list = academies_list.astype(
        input_schemas.academy_master_list
    ).set_index(input_schemas.academy_master_list_index_col).rename(columns={"UKPRN": "Academy UKPRN"})

    print('here')
    group_links = pd.read_csv(
        links_data_path,
        encoding="cp1252",
        index_col=input_schemas.groups_index_col,
        usecols=input_schemas.groups.keys(),
        dtype=input_schemas.groups,
    )[["Group Type", "Group UID"]]
    group_links = group_links[
        group_links["Group Type"].isin(
            ["Single-academy trust", "Multi-academy trust", "Trust"]
        )
    ]
    print('here')
    # remove transitioned schools from academies_list
    mask = (
        academies_list.index.duplicated(keep=False) & ~academies_list["Valid to"].isna()
    )
    print('here')
    academies_list = academies_list[~mask]
    academies_base = academies_list.merge(
        schools.reset_index(), left_index=True, right_on="LA Establishment Number"
    )
    print('here')
    academies = (
        academies_base.merge(census, on="URN", how="left")
        .merge(sen, on="URN", how="left")
        .merge(cdc, on="URN", how="left")
        .merge(aar, on="URN", how="left")
        .merge(ks2, on="URN", how="left")
        .merge(ks4, on="URN", how="left")
        .merge(group_links, on="URN", how="inner")
        .merge(cfo, on="URN", how="left")
    )
    print('here')
    academies["Total Internal Floor Area"] = academies[
        "Total Internal Floor Area"
    ].fillna(academies["Total Internal Floor Area"].median())
    academies["Overall Phase"] = academies.apply(
        lambda df: mappings.map_academy_phase_type(
            df["TypeOfEstablishment (code)"], df["Type of Provision - Phase"]
        ),
        axis=1,
    )
    
    academies["Status"] = academies.apply(
        lambda df: mappings.map_academy_status(
            pd.to_datetime(df["Date joined or opened if in period"]),
            pd.to_datetime(df["Date left or closed if in period"]),
            pd.to_datetime(df["Valid to"]),
            pd.to_datetime(df["OpenDate"]),
            pd.to_datetime(df["CloseDate"]),
            pd.to_datetime(accounts_return_period_start_date),
            pd.to_datetime(academy_year_start_date),
            pd.to_datetime(academy_year_end_date),
        ),
        axis=1,
    )
    
    academies["Period covered by return"] = academies.apply(
        lambda df: mappings.map_academy_period_return(
            pd.to_datetime(df["Date joined or opened if in period"]),
            pd.to_datetime(df["Date left or closed if in period"]),
            pd.to_datetime(academy_year_start_date),
            pd.to_datetime(academy_year_end_date),
        ),
        axis=1,
    )
    
    academies["SchoolPhaseType"] = academies.apply(
        lambda df: mappings.map_school_phase_type(
            df["TypeOfEstablishment (code)"], df["Type of Provision - Phase"]
        ),
        axis=1,
    )

    academies["Finance Type"] = "Academy"

    academies.rename(
        columns={
            "URN_x":"URN",
            "UKPRN_x": "UKPRN",
            "LA (code)": "LA Code",
            "LA (name)": "LA Name",
            "Academy Trust Name": "Trust Name",
            "Academy UKPRN": "Trust UKPRN",
        }
        | config.income_category_map["academies"]
        | config.cost_category_map["academies"],
        inplace=True,
    )

    academies["OfstedLastInsp"] = pd.to_datetime(
        academies["OfstedLastInsp"], dayfirst=True
    )
    academies["London Weighting"] = academies["London Weighting"].fillna("Neither")
    academies["Email"] = ""
    academies["HeadEmail"] = ""
    for category in config.rag_category_settings.keys():
        basis_data = academies[
            (
                "Number of pupils"
                if config.rag_category_settings[category]["type"] == "Pupil"
                else "Total Internal Floor Area"
            )
        ]
        academies = mappings.map_cost_series(category, academies, basis_data)

    academies["Catering staff and supplies_Net Costs"] = (
        academies["Income_Catering services"]
        + academies["Catering staff and supplies_Total"]
    )

    trust_basis_data = (
        academies.sort_values(by="Trust UPIN")[
            ["Number of pupils", "Trust UPIN", "Total Internal Floor Area"]
        ]
        .groupby(["Trust UPIN"])
        .sum()
        .rename(
            columns={
                "Number of pupils": "Total pupils in trust",
                "Total Internal Floor Area": "Total Internal Floor Area in trust",
            }
        )
    )

    central_services = central_services.merge(
        trust_basis_data, on="Trust UPIN", how="left"
    )

    # Apportion central services data based on the given basis of the cost category
    for category in config.rag_category_settings.keys():
        cs_basis_data = central_services[
            (
                "Total pupils in trust"
                if config.rag_category_settings[category]["type"] == "Pupil"
                else "Total Internal Floor Area in trust"
            )
        ]
        central_services = mappings.map_cost_series(
            category, central_services, cs_basis_data
        )

    central_services["Catering staff and supplies_Net Costs"] = (
        central_services["Income_Catering services"]
        + central_services["Catering staff and supplies_Total"]
    )

    # Apportion the central services income fields
    income_cols = central_services.columns[
        central_services.columns.str.startswith("Income_")
    ].values.tolist()

    for income_col in income_cols:
        central_services[income_col] = (
            central_services[income_col] / central_services["Total pupils in trust"]
        )

    academies = academies.merge(
        central_services, on="Trust UPIN", how="left", suffixes=("", "_CS")
    )

    return academies.set_index("URN")

In [ ]:
academies = build_academy_data('data/academy_master_list.csv', 'data/gias_all_links.csv',
                                              current_year, schools, census, sen, cdc, 
                                              academy_ar, ks2, ks4, cfo, central_services)

In [ ]:
#academies.to_csv('output/pre-processing/academies.csv')
academies.sort_index()

Merge required census and cdc data to the maintained schools data set

In [ ]:


def build_federations_data(links_data_path, maintained_schools):
    group_links = pd.read_csv(
        links_data_path,
        encoding="unicode-escape",
        index_col=input_schemas.groups_index_col,
        usecols=input_schemas.groups.keys(),
        dtype=input_schemas.groups,
    )

    federations = maintained_schools[["URN", "LAEstab"]][
        maintained_schools["Federation"] == "Lead school"
    ].copy()

    # join
    federations = federations.join(
        group_links[["Group Name", "Group UID", "Closed Date"]], on="URN"
    )

    # remove federations with an associated closed date
    federations = federations.loc[federations["Closed Date"].isna()]

    # federations with a UID listed in the GIAS groups data are referred to as "Hard" federations
    # while federations not listed in GIAS are referred to as "Soft" federations.
    # Soft federation UIDs are a combination of their URN and LAEstab codes.

    # create mask for soft federations
    mask = federations["Group UID"].isna()

    hard_federations = federations.loc[~mask].copy()
    soft_federations = federations.loc[mask].copy()

    # define members list for hard federations
    group_links["Members"] = group_links.index
    hard_members = group_links[["Members", "Group UID"]].groupby("Group UID").agg(list)

    hard_federations = hard_federations.join(hard_members, on="Group UID")

    # Rename columns
    hard_federations.rename(
        columns={
            "Group Name": "FederationName",
            "Group UID": "FederationUid",
        },
        inplace=True,
    )

    # for the soft federations
    soft_federations["Group UID"] = soft_federations.index.astype(
        str
    ) + soft_federations["LAEstab"].astype(str)

    # Rename columns
    soft_federations.rename(
        columns={
            "Group Name": "FederationName",
            "Group UID": "FederationUid",
        },
        inplace=True,
    )

    return hard_federations, soft_federations


def build_maintained_school_data(
    maintained_schools_data_path,
    links_data_path,
    year,
    schools,
    census,
    sen,
    cdc,
    ks2,
    ks4,
):
    maintained_schools_year_start_date = datetime.date(year - 1, 4, 1)
    maintained_schools_year_end_date = datetime.date(year, 3, 31)

    maintained_schools_list = pd.read_csv(
        maintained_schools_data_path,
        encoding="unicode-escape",
        usecols=input_schemas.maintained_schools_master_list.keys(),
    )
    # remove any DNS schools
    mask = ((maintained_schools_list['Did Not Supply flag'] == '0').values 
            | (maintained_schools_list['Did Not Supply flag'] == 0).values)
    maintained_schools_list = maintained_schools_list.loc[mask]
    maintained_schools_list.replace({'DNS':np.nan}, inplace=True)

    maintained_schools_list = maintained_schools_list.astype(
        input_schemas.maintained_schools_master_list
    ).set_index(input_schemas.maintained_schools_master_list_index_col)
    
    
    maintained_schools = maintained_schools_list.merge(
        schools.reset_index(), left_index=True, right_on="URN"
    )

    maintained_schools = (
        maintained_schools.merge(sen, on="URN", how="left")
        .merge(census, on="URN", how="left")
        .merge(cdc, on="URN", how="left")
        .merge(ks2, on="URN", how="left")
        .merge(ks4, on="URN", how="left")
    )

    maintained_schools["PFI"] = maintained_schools["PFI"].map(
        lambda x: "PFI school" if x == "Y" else "Non-PFI school"
    )

    maintained_schools["Is PFI"] = maintained_schools["PFI"].map(
        lambda x: x == "PFI school"
    )

    maintained_schools["Status"] = maintained_schools.apply(
        lambda df: mappings.map_maintained_school_status(
            df["OpenDate"],
            df["CloseDate"],
            df["Period covered by return (months)"],
            pd.to_datetime(maintained_schools_year_start_date),
            pd.to_datetime(maintained_schools_year_end_date),
        ),
        axis=1,
    )

    maintained_schools["In year balance"] = (
        maintained_schools["Total Income   I01 to I18"]
        - maintained_schools["Total Expenditure  E01 to E32"]
    )

    maintained_schools["Financial Position"] = maintained_schools[
        "In year balance"
    ].map(mappings.map_is_surplus_deficit)

    maintained_schools["SchoolPhaseType"] = maintained_schools.apply(
        lambda df: mappings.map_school_phase_type(
            df["TypeOfEstablishment (code)"], df["Overall Phase"]
        ),
        axis=1,
    )

    maintained_schools["Partial Years Present"] = maintained_schools[
        "Period covered by return (months)"
    ].map(lambda x: x != 12)

    maintained_schools["Did Not Submit"] = maintained_schools[
        "Did Not Supply flag"
    ].map(lambda x: x == 1)

    maintained_schools["Finance Type"] = "Maintained"

    maintained_schools["Email"] = ""
    maintained_schools["HeadEmail"] = ""
    maintained_schools["Trust Name"] = None
    maintained_schools["OfstedLastInsp"] = pd.to_datetime(
        maintained_schools["OfstedLastInsp"], dayfirst=True
    )
    maintained_schools["London Weighting"] = maintained_schools[
        "London Weighting"
    ].fillna("Neither")

    maintained_schools["Income_Direct grants"] = (
        maintained_schools["I01  Funds delegated by the LA"]
        + maintained_schools["I02  Funding for 6th form students"]
        + maintained_schools["I06  Other government grants"]
        + maintained_schools["I07  Other grants and payments"]
    )

    maintained_schools["Income_Targeted grants"] = (
        maintained_schools["I04  Funding for minority ethnic pupils"]
        + maintained_schools["I03  SEN funding"]
        + maintained_schools["I05  Pupil Premium"]
        + maintained_schools[
            "I15  Pupil focussed extended school funding and   or grants"
        ]
    )

    maintained_schools["Income_Total self generated funding"] = (
        maintained_schools["I08  Income from facilities and services"]
        + maintained_schools["I09  Income from catering"]
        + maintained_schools["I10  Receipts from supply teacher insurance claims"]
        + maintained_schools["I11  Receipts from other insurance claims"]
        + maintained_schools["I12  Income from contributions to visits etc "]
        + maintained_schools["I13  Donations and or private funds"]
        + maintained_schools["I17  Community focused school facilities income"]
    )

    maintained_schools["Income_Community grants"] = (
        maintained_schools["I16  Community focussed school funding and   or grants"]
        + maintained_schools["I18  Additional grant for schools"]
    )

    maintained_schools.rename(
        columns={"Period covered by return (months)": "Period covered by return"}
        | config.cost_category_map["maintained_schools"]
        | config.income_category_map["maintained_schools"],
        inplace=True,
    )

    for category in config.rag_category_settings.keys():
        basis_data = maintained_schools[
            (
                "Number of pupils"
                if config.rag_category_settings[category]["type"] == "Pupil"
                else "Total Internal Floor Area"
            )
        ]
        maintained_schools = mappings.map_cost_series(
            category, maintained_schools, basis_data
        )

    maintained_schools["Catering staff and supplies_Net Costs"] = (
        maintained_schools["Income_Catering services"]
        + maintained_schools["Catering staff and supplies_Total"]
    )

    maintained_schools = maintained_schools[maintained_schools.index.notnull()]

    (hard_federations, soft_federations) = build_federations_data(
        links_data_path, maintained_schools
    )

    # Applying federation mappings
    list_of_laestabs = maintained_schools["LAEstab"][
        maintained_schools["Lead school in federation"] != "0"
    ]
    list_of_urns = maintained_schools.index[
        maintained_schools["Lead school in federation"] != "0"
    ]
    lae_ukprn = dict(zip(list_of_laestabs, list_of_urns))

    maintained_schools["Federation Lead School URN"] = maintained_schools[
        "Lead school in federation"
    ].map(lae_ukprn)

    maintained_schools = pd.merge(
        maintained_schools,
        hard_federations[["FederationName"]],
        how="left",
        left_index=True,
        right_index=True,
    )
    maintained_schools.rename(
        columns={"FederationName": "Federation Name"}, inplace=True
    )
    maintained_schools = maintained_schools[~maintained_schools.index.duplicated()]

    return maintained_schools.set_index("URN")


In [ ]:
# Load raw list from CSV
maintained_schools = pre_processing.build_maintained_school_data('data/maintained_schools_master_list.csv','data/gias_all_links.csv',current_year, schools, census, sen, cdc, ks2, ks4)

In [ ]:
maintained_schools.to_csv('output/pre-processing/maintained_schools.csv')
#maintained_schools

In [ ]:
all_schools = pd.concat([academies,maintained_schools])

In [ ]:
all_schools[all_schools["Is PFI"].isna()]

## Federation Capture




In [ ]:
(hard_federations, soft_federations) = pre_processing.build_federations_data('data/gias_all_links.csv', maintained_schools.reset_index())

In [ ]:
hard_federations.to_csv('output/pre-processing/hard_federations.csv')
hard_federations

In [ ]:
soft_federations.to_csv('output/pre-processing/soft_federations.csv')
soft_federations[['LAEstab']]

# Budget Forcast Returns

In [ ]:
year = 2023

bfr_cell_mapping_cols = {'EFALineNo':'Int64','balance_flag':'Int64'}

bfr_sofa_cols = {'TrustUPIN':'Int64','CreatedBy':'string','Category':'string','Title':'string','EFALineNo':'Int64','Y1P1':'float','Y1P2':'float','Y2P1':'float','Y2P2':'float'}
bfr_3y_cols = {'TrustUPIN':'Int64','EFALineNo':'Int64','Y2':'Int64','Y3':'Int64','Y4':'Int64'}


def _calculate_metrics(bfr):
    bfr_metrics = bfr[['TrustUPIN']].copy().set_index('TrustUPIN')
    bfr_metrics['Revenue reserve as percentage of income'] =\
          round(bfr[bfr['Title']=='Revenue reserves'].set_index('TrustUPIN')[['Y1']]
                /bfr[bfr['Title']=='Total income'].set_index('TrustUPIN')[['Y1']]*100,1)
    bfr_metrics['Staff costs as percentage of income'] =\
          round(bfr[bfr['Title']=='Staff costs'].set_index('TrustUPIN')[['Y1']]
                /bfr[bfr['Title']=='Total income'].set_index('TrustUPIN')[['Y1']]*100,1)
    bfr_metrics['Expenditure as percentage of income'] =\
          round(bfr[bfr['Title']=='Total expenditure'].set_index('TrustUPIN')[['Y1']]
                /bfr[bfr['Title']=='Total income'].set_index('TrustUPIN')[['Y1']]*100,1)
    bfr_metrics['percent self-generated income'] =\
          round(bfr[bfr['Title']=='Self-generated income'].set_index('TrustUPIN')[['Y1']]/
                (bfr[bfr['Title']=='Self-generated income'].set_index('TrustUPIN')[['Y1']] +
                  bfr[bfr['Title']=='Grant funding'].set_index('TrustUPIN')[['Y1']])*100,0)
    bfr_metrics['percent grant funding'] = 100 - bfr_metrics['percent self-generated income']
    return bfr_metrics

def _calculate_slopes(matrix):
    x = np.array([1,2,3,4,5,6])
    x_bar = 3.5
    x_x_bar = x - x_bar
    y_bar = np.mean(matrix, axis=1)
    y_y_bar = matrix - np.vstack(y_bar)
    slope_array = np.sum(x_x_bar*y_y_bar,axis=1)/np.sum(x_x_bar**2)
    return slope_array

def _assign_slope_flag(df):
    percentile_10 = np.nanpercentile(df['slope'].values, 10)
    percentile_90 = np.nanpercentile(df['slope'].values, 90)
    df['slope_flag'] = 0
    df.loc[df['slope'] < percentile_10, 'slope_flag'] = -1
    df.loc[df['slope'] > percentile_90, 'slope_flag'] = 1
    return df


def _slope_analysis(bfr_dataframe, academies_y2, academies_y1):

    year_columns = ['Y-2','Y-1','Y1','Y2','Y3','Y4']
    bfr_revenue_reserves = bfr_dataframe[bfr_dataframe['Title']=='Revenue reserves']
    bfr_pupil_numbers = bfr_dataframe[bfr_dataframe['Title']=='Pupil numbers']

    

    # TODO need to add in historic data to this, filling in fake values for now
    bfr_revenue_reserves = pd.merge(
        bfr_revenue_reserves, 
        academies_y2[['Trust UPIN','Trust Balance']].rename(columns={
            'Trust UPIN':'TrustUPIN',
            'Trust Balance':'Y-2'
            }).drop_duplicates(), how='left', on='TrustUPIN')
    
    bfr_revenue_reserves = pd.merge(
        bfr_revenue_reserves, 
        academies_y1[['Trust UPIN','Trust Balance']].rename(columns={
                'Trust UPIN':'TrustUPIN',
                'Trust Balance':'Y-1'
                }).drop_duplicates(), how='left', on='TrustUPIN')
    
    bfr_pupil_numbers = pd.merge(
        bfr_pupil_numbers, 
        academies_y2[['Trust UPIN','Number of pupils']].rename(columns={
            'Trust UPIN':'TrustUPIN',
            'Number of pupils':'Y-2'
            }).groupby('TrustUPIN').agg(sum), how='left', on='TrustUPIN')
    
    bfr_pupil_numbers = pd.merge(
        bfr_pupil_numbers, 
        academies_y2[['Trust UPIN','Number of pupils']].rename(columns={
            'Trust UPIN':'TrustUPIN',
            'Number of pupils':'Y-1'
            }).groupby('TrustUPIN').agg(sum), how='left', on='TrustUPIN')


    # convert to matrix
    matrix_revenue_reserves = bfr_revenue_reserves[year_columns].values.astype(float)
    matrix_pupil_numbers = bfr_pupil_numbers[year_columns].values.astype(float)

    matrix_revenue_reserves_per_pupil = matrix_revenue_reserves/matrix_pupil_numbers

    # determine associated slopes
    bfr_revenue_reserves['slope'] = _calculate_slopes(matrix_revenue_reserves)

    bfr_revenue_reserves_per_pupil = bfr_revenue_reserves[['CreatedBy','Category','Title','EFALineNo']].copy()
    bfr_revenue_reserves_per_pupil['slope'] = _calculate_slopes(matrix_revenue_reserves_per_pupil)
    for i in range(len(year_columns)):
        bfr_revenue_reserves_per_pupil[year_columns[i]] = matrix_revenue_reserves_per_pupil.T[i]


    # flag top 10% and bottom 90% percent of slopes with -1 and 1 respectively
    bfr_revenue_reserves = _assign_slope_flag(bfr_revenue_reserves)
    bfr_revenue_reserves_per_pupil = _assign_slope_flag(bfr_revenue_reserves_per_pupil)

    return bfr_revenue_reserves, bfr_revenue_reserves_per_pupil

def _volatility_analysis(bfr):
    bfr['volatility'] = (bfr['Trust Balance'] - bfr['Y1P2'])/abs(bfr['Trust Balance'])

    volatility_conditions = [(bfr['volatility'] <= -0.05),
                            (bfr['volatility'] <= 0.05),
                            (bfr['volatility'] <= 0.1),
                            (bfr['volatility'] > 0.1)]
    volatility_messages = ["AR below forecast", 
                        "stable forecast", 
                        "AR above forecast", 
                        "AR significantly above forecast"]

    bfr['volatility_status'] = np.select(volatility_conditions, volatility_messages, default='')
    return bfr

def build_bfr_data(bfr_sofa_data_path,bfr_3y_data_path, academies_y2, academies_y1, academies):

    bfr_sofa = pd.read_csv(
        bfr_sofa_data_path,
        encoding='unicode-escape',
        dtype=bfr_sofa_cols,
        usecols=bfr_sofa_cols.keys(),
    )

    bfr_3y = pd.read_csv(
        bfr_3y_data_path,
        encoding='unicode-escape',
        dtype=bfr_3y_cols,
        usecols=bfr_3y_cols.keys(),
    )    


    # remove unused metrics
    bfr_sofa = bfr_sofa[bfr_sofa['EFALineNo'].isin([298,430,335,380,211,220,199,200,205,210,999])]

    self_gen_income = bfr_sofa[
        bfr_sofa['EFALineNo'].isin([211,220])
        ].groupby('TrustUPIN')[['Y1P1','Y1P2','Y2P1','Y2P2']].sum().reset_index()
    self_gen_income['Title'] = 'Self-generated income'

    grant_funding = bfr_sofa[
        bfr_sofa['EFALineNo'].isin([199,200,205,210])
        ].groupby('TrustUPIN')[['Y1P1','Y1P2','Y2P1','Y2P2']].sum().reset_index()
    grant_funding['Title'] = 'Grant funding'

    bfr_sofa = bfr_sofa[~bfr_sofa['EFALineNo'].isin([211,220,199,200,205,210])]
    bfr_sofa = pd.concat([bfr_sofa, self_gen_income, grant_funding])
    bfr_sofa['Title'].replace({
        'Balance c/f to next period ':'Revenue reserves',
        'Pupil numbers (actual and estimated)':'Pupil numbers',
        'Total revenue expenditure':'Total expenditure',
        'Total revenue income':'Total income','Total staff costs':'Staff costs'
        }, inplace=True)
    bfr_sofa['Y1'] = bfr_sofa['Y1P1'] + bfr_sofa['Y1P2']
    bfr_sofa.drop_duplicates(inplace=True)
    
    bfr_3y['EFALineNo'].replace({2980:298,4300:430,3800:380,9000:999}, inplace=True)
    bfr_3y = bfr_3y[bfr_3y['EFALineNo'].isin([298,430,335,380,999])]
    bfr_3y.drop_duplicates(inplace=True)
    

    bfr = pd.merge(bfr_sofa, bfr_3y, how='left', on=('TrustUPIN','EFALineNo'))
    
    # get trust metrics
    bfr_metrics = _calculate_metrics(bfr)
    # Slope analysis
    bfr_revenue_reserves, bfr_revenue_reserves_per_pupil = _slope_analysis(bfr, academies_y2, academies_y1)

    # volatility analysis
    bfr = pd.merge(bfr, academies[['Trust UPIN','Trust Balance']].rename(
        columns={'Trust UPIN': 'TrustUPIN'}), how='left', on='TrustUPIN')
    bfr = _volatility_analysis(bfr)
    
    bfr_metrics.drop_duplicates(inplace=True)
    
    use_columns = ["Y-2","Y-1","Y1","Y2","Y3","slope","slope_flag"]
    
    bfr_revenue_reserves.drop_duplicates(inplace=True)
    bfr_revenue_reserves = bfr_revenue_reserves[use_columns]
    bfr_revenue_reserves.rename(columns={
        "Y-2":"revenue_reserves_year_-2",
        "Y-1":"revenue_reserves_year_-1",
        "Y1":"revenue_reserves_year_0",
        "Y2":"revenue_reserves_year_1",
        "Y3":"revenue_reserves_year_2",
        "slope":"revenue_reserves_slope",
        "slope_flag":"revenue_reserves_slope_flag"}, inplace=True)
    
    
    bfr_revenue_reserves_per_pupil.drop_duplicates(inplace=True)
    bfr_revenue_reserves_per_pupil = bfr_revenue_reserves_per_pupil[use_columns]
    bfr_revenue_reserves_per_pupil.rename(columns={
        "Y-2":"revenue_reserves_year_per_pupil_-2",
        "Y-1":"revenue_reserves_year_per_pupil_-1",
        "Y1":"revenue_reserves_year_per_pupil_0",
        "Y2":"revenue_reserves_year_per_pupil_1",
        "Y3":"revenue_reserves_year_per_pupil_2",
        "slope":"revenue_reserves_year_per_pupil_slope",
        "slope_flag":"revenue_reserves_year_per_pupil_slope_flag"}, inplace=True)
    
    bfr_metrics = pd.merge(bfr_metrics, bfr_revenue_reserves, left_index=True, right_index=True)
    bfr_metrics = pd.merge(bfr_metrics, bfr_revenue_reserves_per_pupil, left_index=True, right_index=True)
    return bfr_metrics, bfr

In [ ]:
bfr_metrics, bfr = build_bfr_data('data/BFR_SOFA_raw.csv','data/BFR_3Y_raw.csv', academies.copy().reset_index(), academies.copy().reset_index(), academies.copy().reset_index())

### Timing Keep at the bottom

In [ ]:
print(f'Processing Time: {time.time() - start_time} seconds')

There are 327 duplicates in the academies outputs, and 346 in the maintained (excluding federations)


academy_ar has 10444 entries, 148 of the urns in this list are duplicated, though they look to be schools which have changed from SAT to MAT

The academies_list containes duplicated LAEstabs due to schools transitioning between SAT / MAT etc.

maintained_schools has 10650 entries, 347 of which are nulls. These can just be dropped

There are a few duplicates in federation data as well: